In [ ]:
import pandas as pd
import numpy as np

#use this colab tool to upload files into colab notebook
# from google.colab import files
# file = files.upload()

# Dataset used here is the combination iof NSL-KDD train and test text files
#seting feature columns using numpy array

feature_col = np.arange(41)
train_ds = pd.read_csv("/content/drive/My Drive/RM3/KDDTotal.txt", usecols=(feature_col), dtype= str, encoding= "utf-8").values





In [ ]:
#tools for preprocessing from sklearn

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

# Encoding categeroical data 
# Note: Datasets used have 43 columns :Extra [:41] the 43rd column is attack rate which dont support the model 

# fit_transform is used to transform dataset after encoding

for c in range(1,4):
  train_ds[:,c] = LabelEncoder().fit_transform(train_ds[:,c])

print(train_ds)
# Now all features are in numeric 

[['0' 2 44 ... '0.00' '0.00' '0.00']
 ['0' 1 49 ... '1.00' '0.00' '0.00']
 ['0' 1 24 ... '0.01' '0.00' '0.01']
 ...
 ['0' 1 24 ... '0.00' '0.07' '0.07']
 ['0' 2 12 ... '0.00' '0.00' '0.00']
 ['0' 1 57 ... '0.00' '0.44' '1.00']]


In [ ]:
# Here all numeric features converted into Float

train_ds=train_ds.astype(np.float)

# Here we standardizing training features

# sc = StandardScaler()
# sc.fit(train_ds)
# ds_normalized = sc.transform(train_ds)

# MinMax scalling 

MMS = MinMaxScaler(feature_range = (0,1))
MMS.fit(train_ds)

# target column for training is taken from the dataset

target_tn=pd.read_csv("/content/drive/My Drive/RM3/KDDTotal.txt", usecols=([41]), dtype= str, encoding= "utf-8").values

# As dataset contains #many number of target features
# For the usability in NIDS it should be able to recognize kind of traffic
# To classify the traffic target labels to be as anomaly or normal traffic

for i in range(0,len(target_tn)):
  if target_tn[i] != 'normal':
    target_tn[i] = 'anomaly'
target_tn

#Creating training pipeline

ds_normalized = MMS.transform(train_ds)
target_label = LabelEncoder().fit_transform(target_tn)

# Feature engineering of training data is finished



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Splitting test, train pipelines with features(x), target(y)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(ds_normalized, target_label, test_size=.2, random_state=40)
print(y_train)

[1 0 1 ... 0 0 1]


In [ ]:
# using classifier and ensemble classifiers from sklearn

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# building 3 base learners as Logistic regression, RFC, Naive Bayes

clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

In [ ]:
 # 'eclf' is hard voting ensembled classifier 
import time
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

start = time.time()

eclf1 = eclf1.fit(x_train, y_train)

stop = time.time()
print(f"trainig time:{stop -start}s")

print(eclf1.predict(x_test))
score = eclf1.score(x_test,y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


trainig time:12.731932163238525s
[0 1 1 ... 0 1 1]


In [ ]:
# Here is accuracy of ensembled model

print("Score of model on new data", score)

Score of model on new data 0.9498720711015352


In [ ]:
clf1 = clf1.fit(x_train, y_train)

score_1 = clf1.score(x_test,y_test)
print(score_1)

0.9352612442768651


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
clf2 = clf2.fit(x_train, y_train)

score_2 = clf2.score(x_test,y_test)
print(score_2)

0.9947481820630218


In [ ]:
clf3 = clf3.fit(x_train, y_train)

score_3 = clf3.score(x_test,y_test)
print(score_3)

0.8717344465391866


ROUGH WORK

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris


iris = load_iris()
ds_normalized = iris.data[:, :]  
target_label = iris.target


scores = cross_val_score(eclf1, iris.data, iris.target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy: 0.97 (+/- 0.04)


In [ ]:
clf3.get_params()

{'priors': None, 'var_smoothing': 1e-09}

{'priors': None, 'var_smoothing': 1e-09}

In [ ]:
eclf1.get_params()

{'estimators': [('lr',
   LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                      intercept_scaling=1, l1_ratio=None, max_iter=100,
                      multi_class='multinomial', n_jobs=None, penalty='l2',
                      random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                      warm_start=False)),
  ('rf',
   RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                          criterion='gini', max_depth=None, max_features='auto',
                          max_leaf_nodes=None, max_samples=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=50,
                          n_jobs=None, oob_score=False, random_state=1, verbose=0,
                          warm_start=False)),
  ('gnb', GaussianNB(priors=None, var_smoothing=1e-0

{'estimators': [('lr',
   LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                      intercept_scaling=1, l1_ratio=None, max_iter=100,
                      multi_class='multinomial', n_jobs=None, penalty='l2',
                      random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                      warm_start=False)),
  ('rf',
   RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                          criterion='gini', max_depth=None, max_features='auto',
                          max_leaf_nodes=None, max_samples=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=50,
                          n_jobs=None, oob_score=False, random_state=1, verbose=0,
                          warm_start=False)),
  ('gnb', GaussianNB(priors=None, var_smoothing=1e-0